In [ ]:
%%capture --no-display

# Librairies pour compilation_donnees
# ! pip install pandas py7zr

In [ ]:
%%capture --no-display

# Librairies et setup pour effectuer le scrapping
# ! pip install pytest-playwright playwright_stealth asyncio
# ! sudo playwright install-deps
# ! playwright install

# 1 - Traitement et importation des données

## Données tabulaires

Toutes les fonctions d'extraction de données brutes se trouvent dans le script `scripts/compilation_donnees.py`. Les fonctions `importer_tout()` et `importer_locale(date)` permettent d'importer toutes les sources.

In [1]:
from scripts.compilation_donnees import *

## Importer les données depuis les APIs (plus lent)
# df_SAE, df_SAE_2011, df_dep, df_finess, df_drees, df_pauv = importer_tout() 

## Importer une sauvegarde locale récente des données
df_SAE, df_SAE_2011, df_dep, df_finess, df_drees, df_pauv = importer_locale("2025-12-12-10-24")

Travail sur les données DREES pour réaliser des évolutions dans le temps au niveau national : juste présenter les données comme y'a déjà publier sur la DREES 

In [ ]:
df_drees_filtre = df_drees[df_drees["zone_geo"] == "France entière"]
df_drees_filtre.plot(x="annee", y=["TOT_IVG", "IVG_HOSP_INS", "IVG_HOSP_MED"], kind="line", marker='o', figsize=(10,6), title="Évolution nombre total d'IVG en France de 2016 à 2024")

La hausse du nombre d’IVG depuis 2022 se poursuit, après la nette baisse en 2020 et 2021 en lien avec la pandémie de Covid-19.
Le nombre d’IVG a tendance a augmenté depuis 2017. Cette hausse est interrompu en 2020 et 2021, où le nombre d’IVG connait une chute en lien avec la pandémie de Covid-19. Depuis 2022, ce chiffre augmente de nouveau, et plus rapidement que précédemment. 
Le nombre d’IVG tardive suit une évolution similaire. Néanmoins, la hausse est plus faible que celle du nombre d’IVG totale entre 2023 et 2024, ce qui peut indiquer un changement des pratiques, qui suit le changement législatif de 2022. En effet, depuis 2022, il est légal de recourir à une IVG après 14 semaines de grossesse, deux semaines de plus qu’auparavant. 

Pour finess

In [ ]:
df_nbcentre = pd.crosstab(df_finess["14"], df_finess["19"])

df_nbcentre.index.name = "département"
df_nbcentre.rename(columns={
    'Centre de santÃ© sexuelle': 'Centre de sante sexuelle', 
    "Centre gratuit d'information de dÃ©pistage et de diagnostic" : 'CeGIDD'
}, inplace=True)
df_nbcentre = df_nbcentre[["Centre de sante sexuelle", "CeGIDD"]]

df_nbcentre.head(5)
# à normaliser par nombre d'habitant, pour 1000 habitant (pr avoir un chiffre interprétable)

Pour départements

In [ ]:
df_dep.head(5)

Pour la SAE

In [6]:
df_SAE_2011.rename(columns={'DEP': 'code_dep'}, inplace=True)
df_SAE_2011.head()

,FI,RS,nb_ivg,nb_ivg_medic,delai_moy_pec_ivg,CATEGORIE,code_dep
0,010007300,CLINIQUE AMBULATOIRE CENDANEG,0,0,sans objet,Etablissement de soins chirurgicaux,01
1,010007987,CH HAUTEVILLE UNITE INTERDEPT.,0,0,sans objet,Centre hospitalier (CH),01
2,010008407,CH DU HAUT BUGEY,0,0,sans objet,Centre hospitalier (CH),01
3,010780054,CH DE BOURG-EN-BRESSE FLEYRIAT,646,474,10,Centre hospitalier (CH),01
4,010780062,CH DOCTEUR RECAMIER BELLEY,115,72,6,Centre hospitalier (CH),01


In [ ]:
df_SAE.rename(columns={'fi': 'FI', 'dep': 'code_dep'}, inplace=True)

# nb centre qui prend en charge les IVG par département 
df_prise_en_charge_dep = (
    df_SAE
    .groupby('code_dep')[['PRIS', 'CONV']]
    .sum()
    .reset_index()
)
df_dep_ivg = pd.merge(df_prise_en_charge_dep, df_dep, on=['code_dep'], how='outer') # on merge les deux bases 

df_dep_ivg['département'] = (
    df_dep_ivg['département']
        .str.replace('-', ' ', regex=False)
        .str.upper()
) # modifier la typo de la colonne Département 

df_dep_ivg = pd.merge(df_dep_ivg, df_nbcentre, on=['département'], how='outer')

In [ ]:
df_ivg_sans_tard = (
    df_SAE
    .loc[
        (df_SAE['IVG1214'].fillna(0) == 0) &
        (df_SAE['IVG1516'].fillna(0) == 0)
    ]
    .groupby('code_dep')
    .size()
    .reset_index(name='nb_hopitaux_sans_ivg_tard')
)
df_dep_ivg = pd.merge(df_dep_ivg, df_ivg_sans_tard, on=['code_dep'], how='outer')
df_dep_ivg['nb_hopitaux_sans_ivg_tard'] = df_dep_ivg['nb_hopitaux_sans_ivg_tard'].fillna(0)

In [ ]:
df_dep_ivg["centres_total"] = df_dep_ivg["Centre de sante sexuelle"] + df_dep_ivg["CeGIDD"]

def par100k(x):
    return (x/df_dep_ivg["femmes"])*100000

df_dep_ivg["PRIS_pour100kfe"] = par100k(df_dep_ivg["PRIS"])
df_dep_ivg["centres_pour100kfe"] = par100k(df_dep_ivg["Centre de sante sexuelle"])
df_dep_ivg["cegidd_pour100kfe"] = par100k(df_dep_ivg["CeGIDD"])
df_dep_ivg["centres_total_pour100kfe"] = par100k(df_dep_ivg["centres_total"])
df_dep_ivg["hop_sansIVGtard_pour100kfe"] = par100k(df_dep_ivg["nb_hopitaux_sans_ivg_tard"])
valeur_min = df_dep_ivg["femmes"].min()

print(valeur_min) # 27982 < 100 000 : donc on conserve pr avoir un chiffre interprétable mais pas réaliste pr certains départements

Pour le taux de pauvreté,

In [ ]:
df_pauv.columns = ['code_dep', 'Département', 'taux_pauvrete']
df_pauv.head()

Mettre carte récap avec "toutes" les données

## Web Scrapping - Doctolib

Pour avoir un **indice de la disponibilité des IVGs en France**, nous allons utiliser des données issues du site de prise de rendez-vous médicales le plus utilisé en France, **Doctolib**. Par département et spécialisation IVG, les données extraites sont le nombre de localisations différentes proposant un RDV IVG dans un délai de 14, 30 et 90 jours. Ces données sont accessibles à tous gratuitement en utilisant leur moteur de recherche et des filtres (par [ex](https://www.doctolib.fr/ivg-medicamenteuse/paris?availabilitiesBefore=90&regulationSector%5B%5D=CONTRACTED_1&regulationSector%5B%5D=CONTRACTED_1_WITH_EXTRA&regulationSector%5B%5D=CONTRACTED_1_WITH_OPTAM&regulationSector%5B%5D=CONTRACTED_1_WITH_OPTAM_CO&regulationSector%5B%5D=CONTRACTED_2&regulationSector%5B%5D=CONTRACTED_2_WITH_OPTAM&regulationSector%5B%5D=CONTRACTED_2_WITH_OPTAM_CO&regulationSector%5B%5D=CONTRACTED&regulationSector%5B%5D=CONTRACTED_WITH_EXTRA&regulationSector%5B%5D=ORGANIZATION_CONTRACTED)).

Pour la mise en oeuvre, le script est `scrapper_doctolib.py` qui crée un .csv à l'aboutissement. Le script s'appuie sur la librairie `Playwright` permet de faire du web scrapping sur des sites dynamiques (comme Doctolib qui utilise react et javascript) en utilisant des navigateurs "headless" dans l'arrière plan. Avant de finir sur cette librairie, d'autres options ont été tentées : `BeautifulSoup` (pour adapté au parsing des sites statiques) et `Selenium` (une autre librairie de "headless browsers"). Cependant, `Playwright` a était le plus agile pour la gestion du chargement de la page. De plus, `asyncio` est utilisé pour faire du parallélisme et réduire le temps de scrapping.

> Point législatif
> 
> [Source (CNIL) : ](https://www.cnil.fr/fr/focus-interet-legitime-collecte-par-moissonnage)

Exemple des données obtenues (format dataframe pandas) suite au scraping (environ une demi-heure) :

In [2]:
dictionnaire_departements = {
    "ain": "01","aisne": "02","allier": "03","alpes-de-haute-provence": "04","hautes-alpes": "05",
    "alpes-maritimes": "06","ardeche": "07","ardennes": "08","ariege": "09","aube": "10",
    "aude": "11","aveyron": "12","bouches-du-rhone": "13","calvados": "14","cantal": "15",
    "charente": "16","charente-maritime": "17","cher": "18","correze": "19",
    "corse-du-sud": "2A","haute-corse": "2B","cote-d-or": "21","cotes-d-armor": "22",
    "creuse": "23","dordogne": "24","doubs": "25","drome": "26","eure": "27","eure-et-loir": "28",
    "finistere": "29","gard": "30","haute-garonne": "31","gers": "32","gironde": "33","herault": "34",
    "ille-et-vilaine": "35","indre": "36","indre-et-loire": "37","isere": "38","jura": "39",
    "landes": "40", "loir-et-cher": "41","loire": "42","haute-loire": "43","loire-atlantique": "44",
    "loiret": "45","lot": "46","lot-et-garonne": "47","lozere": "48","maine-et-loire": "49","manche": "50",
    "marne": "51","haute-marne": "52","mayenne": "53","meurthe-et-moselle": "54","meuse": "55",
    "morbihan": "56","moselle": "57","nievre": "58","nord": "59","oise": "60","orne": "61",
    "pas-de-calais": "62","puy-de-dome": "63","pyrenees-atlantiques": "64","hautes-pyrenees": "65",
    "pyrenees-orientales": "66","bas-rhin": "67","haut-rhin": "68","rhone": "69","haute-saone": "70",
    "saone-et-loire": "71","sarthe": "72","savoie": "73","haute-savoie": "74","paris": "75","seine-maritime": "76",
    "seine-et-marne": "77","yvelines": "78","deux-sevres": "79","somme": "80","tarn": "81","tarn-et-garonne": "82",
    "var": "83","vaucluse": "84","vendee": "85","vienne": "86","haute-vienne": "87","vosges": "88",
    "yonne": "89","territoire-de-belfort": "90","essonne": "91","hauts-de-seine": "92",
    "seine-saint-denis": "93","val-de-marne": "94","val-d-oise": "95"}
df_doctolib = pd.read_csv("./donnees/doctolib_2025-11-20-14-56.csv")
df_doctolib['code_dep'] = df_doctolib['departement'].map(dictionnaire_departements).astype(str).str.zfill(2)
df_doctolib.head()

,departement,specialite,trimestre,mois,deuxSemaines,code_dep
0,ain,ivg-chirurgicale,0,0,0,01
1,ain,ivg-medicale-et-chirurgicale,1,1,1,01
2,ain,ivg-medicamenteuse,87,77,69,01
3,aisne,ivg-chirurgicale,0,0,0,02
4,aisne,ivg-medicale-et-chirurgicale,0,0,0,02


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

In [ ]:
url = "https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb" 
geo = gpd.read_file(url)

In [ ]:
geo['code_dep'] = geo['code'].astype(str).str.zfill(2)
gdf = gpd.merge(df_doctolib, geo, on='code_dep', how='left')

In [ ]:
gdf.head()

In [ ]:
aggregated = gdf.groupby(['code', 'nom', 'geometry'], as_index=False).agg({
    'trimestre': 'sum',
    'mois': 'sum',
    'deuxSemaines': 'sum'
})
aggregated.head()

In [ ]:
import geopandas as gpd
import folium
import mapclassify
import matplotlib

gdf_aggregated = gpd.GeoDataFrame(aggregated)

gdf_aggregated.explore(
    column="trimestre",
    scheme="quantiles", 
    legend=True,
    tooltip=["nom", "trimestre", "mois", "deuxSemaines"],
    cmap="Reds",
    tiles="CartoDB positron",
    k=10,                 
    legend_kwds=dict(colorbar=True)
)


Cette carte présente les taux de disponibilité d’IVG dans un délai de moins de 3 mois via doctolib dans un département. Les disparités territoriales sont importantes. Certains départements comme l’Ile-de-France et …. ont une plus grande offre de rendez-vous médicaux pour réaliser une IVG, quelque soit le professionnel de santé. D’autres, comme …, ont seulement … offre pour 1000 femmes. Ces constats révèlent des offres différenciées selon les départements, et donc des inégalités territoriales pour recourir à une IVG. 

In [ ]:
gdf_aggregated = pd.merge(aggregated, df_dep, left_on='code', right_on='DEP', how='left')
gdf_aggregated.drop(['REG', 'Région', 'DEP', 'Département'], axis=1)

In [ ]:
gdf_aggregated = gdf_aggregated.rename(columns={'PMUN': 'population'})
gdf_aggregated['trimestre_norm'] = gdf_aggregated['trimestre']/(gdf_aggregated['population']/2)*100000
gdf_aggregated['deuxSemaines_norm'] = gdf_aggregated['deuxSemaines']/(gdf_aggregated['population']/2)*100000
gdf_aggregated['mois_norm'] = gdf_aggregated['mois']/(gdf_aggregated['population']/2)*100000


In [ ]:
gdf_aggregated = gpd.GeoDataFrame(gdf_aggregated)

gdf_aggregated.explore(
    column="trimestre_norm",
    scheme="quantiles", 
    legend=True,
    tooltip=["nom", "trimestre_norm", "mois_norm", "deuxSemaines_norm"],
    cmap="Reds",
    tiles="CartoDB positron",
    k=10,                 
    legend_kwds=dict(colorbar=True)
)
